In [257]:
import functools
from vantage6.client import Client
from vantage6.client.filters import post_filtering

In [258]:
class NewClient(Client):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.use_master_container = False
        self.collaboration = None
        self.image = None
        
    def wait_and_collect_results(client, task_id):
        task = client.get_task(task_id)
        while not task.get("complete"):
            task = client.get_task(task_id)
            info("Waiting for results")
            time.sleep(1)

        info("Obtaining results")
        results = client.get_results(task_id=task.get("id"))
        return results
    
    def call(self, method, input_, master=False):
        assert self.collaboration
        assert self.image
        pass
    
    @post_filtering()
    def view_collaboration_nodes(self, collaboration_id=None):
        col_id = collaboration_id or self.collaboration
        assert col_id, "collaboration id missing"
        return client.request(f'collaboration/{col_id}/node')
        
    @post_filtering()
    def view_my_nodes(self):
        return client.request(f'organization/{self.whoami.organization_id}/node')
        
    def view_server_version(self):
        return client.request('version')
    
    @post_filtering(iterable=False)
    def view_my_organization(self):
        return client.request(f'organization/{self.whoami.organization_id}')
    
    @post_filtering()
    def view_my_collaborations(self):
        return client.request(f'organization/{self.whoami.organization_id}/collaboration')
        
    @post_filtering(iterable=False)
    def view_my_details(self, fields=None):
        return client.request(f'user/{self.whoami.id_}')
        
    @post_filtering()
    def view_rules(self, fields=None, filters=None):
        return client.request('rule')

In [259]:
client = NewClient("http://localhost", 5000, "/api")
client.authenticate("root", "root")

In [260]:
client.setup_encryption(None)

In [261]:
client.view_server_version()

{'version': '2.0.0a1'}

In [262]:
client.view_my_collaborations(fields=('id','name'))

[]

In [263]:
client.view_my_organization(fields=('id', 'name'))

{'id': 4, 'name': 'root'}

In [264]:
client.view_my_details(fields=('id', 'username','email', 'roles'))

{'id': 14,
 'username': 'root',
 'email': 'root@domain.ext',
 'roles': [{'id': 3,
   'link': '/api/role/3',
   'methods': ['GET', 'PATCH', 'DELETE']}]}

In [265]:
client.view_collaboration_nodes(1, fields=('id', 'name', 'status'))

[{'id': 7, 'name': 'IKNL - ZEPPELIN Node', 'status': 'offline'},
 {'id': 8, 'name': 'Small Organization - ZEPPELIN Node', 'status': 'offline'},
 {'id': 9, 'name': 'Big Organization - ZEPPELIN Node', 'status': 'offline'}]

In [266]:
client.view_my_nodes(fields=('id','name',))

[]

In [267]:
client.view_my_nodes(fields=('ip','name', 'status'))

[]

In [268]:
client.view_rules(fields=('id','scope'), filters=(('operation', 'VIEW'),))

[{'id': 1, 'scope': 'GLOBAL'},
 {'id': 2, 'scope': 'ORGANIZATION'},
 {'id': 9, 'scope': 'GLOBAL'},
 {'id': 10, 'scope': 'ORGANIZATION'},
 {'id': 14, 'scope': 'GLOBAL'},
 {'id': 15, 'scope': 'ORGANIZATION'},
 {'id': 16, 'scope': 'COLLABORATION'},
 {'id': 20, 'scope': 'GLOBAL'},
 {'id': 21, 'scope': 'ORGANIZATION'},
 {'id': 28, 'scope': 'GLOBAL'},
 {'id': 29, 'scope': 'ORGANIZATION'},
 {'id': 30, 'scope': 'GLOBAL'},
 {'id': 31, 'scope': 'ORGANIZATION'},
 {'id': 32, 'scope': 'OWN'},
 {'id': 41, 'scope': 'GLOBAL'},
 {'id': 42, 'scope': 'ORGANIZATION'}]

In [276]:
client.whoami.id_

14

In [ ]:
@post_filtering
def update_my_user(firstname=None, lastname=None, password=None, roles=None, rules=None, organization=None):
    id_ = client.whoami.id_
    user = client.request(f'user/{client.whoami.id_}', method='patch', json={
        "firstname": "New-root",
        "lastname": "Baba",
    })

In [278]:
client.request(f'user/{client.whoami.id_}', method='patch', json={
    "firstname": "New-root",
    "lastname": "Baba",
})

{'ip': '127.0.0.1',
 'id': 14,
 'firstname': 'New-root',
 'lastname': 'Baba',
 'last_seen': '2021-01-13T15:29:18.270138+00:00',
 'organization': {'id': 4,
  'link': '/api/organization/4',
  'methods': ['PATCH', 'GET']},
 'username': 'root',
 'rules': None,
 'email': 'root@domain.ext',
 'status': None,
 'roles': [{'id': 3,
   'link': '/api/role/3',
   'methods': ['GET', 'PATCH', 'DELETE']}],
 'type': 'user'}